<div align="center">

# 🍬 Sweeta - SORA 2 Watermark Remover

### Remove watermarks from SORA 2 video generations

[![GitHub](https://img.shields.io/badge/GitHub-kuberwastaken/sweeta-pink?logo=github)](https://github.com/kuberwastaken/sweeta)
[![License](https://img.shields.io/badge/License-Apache%202.0-pink.svg)](https://github.com/kuberwastaken/sweeta/blob/main/LICENSE)

---

**What this does:** Removes watermarks from SORA 2 generated videos using AI

**Runtime:** ~5-15 minutes depending on video length
---

</div>


## ⚠️ Important Disclaimer

**FOR EDUCATIONAL AND RESEARCH PURPOSES ONLY**

This tool is provided to:
- Demonstrate AI capabilities in computer vision
- Encourage OpenAI to implement stronger watermarking
- Research watermark removal techniques

**Please use responsibly and ethically. Respect content creator rights and intellectual property.**

---


## Instructions

Follow these steps in order:

1. **Step 1:** Enable GPU runtime (Runtime → Change runtime type → T4 GPU)
2. **Step 2:** Run the setup cell (installs conda)
3. **Step 3:** Clone the Sweeta repository
4. **Step 4:** Upload your SORA 2 video
5. **Step 5:** Process the video
6. **Step 6:** Download your watermark-free video

⏱️ **Total time:** ~5-15 minutes

---


## Step 0: Check GPU Status

Let's make sure you have GPU enabled! This should show your GPU info.


In [ ]:
# Check if GPU is available
!nvidia-smi -L
print("\n✅ GPU detected! You're good to go.")
print("\n⚠️ If you see an error above, go to: Runtime → Change runtime type → T4 GPU")


## Step 1: Install Conda

This installs conda (a package manager) which is needed for the project.

**Time:** ~30 seconds

**⚠️ Note:** The kernel will restart after this step. That's normal!


In [ ]:
# Install condacolab (required for conda in Colab)
!pip install -q condacolab
import condacolab
condacolab.install()

print("Conda installed successfully!")
print("Kernel will restart automatically...")


## Step 2: Clone Sweeta Repository

This downloads the Sweeta code from GitHub.

In [ ]:
import os

# Clone the repository
if not os.path.exists('/content/Sweeta'):
    print(" Cloning Sweeta repository...")
    !git clone https://github.com/kuberwastaken/Sweeta.git
    print("\n✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists!")

# Change to the project directory
%cd /content/Sweeta
print("\n📂 Current directory:", os.getcwd())


## Step 3: Install Dependencies

This sets up the Python environment and downloads the AI model.

**What's happening:**
- Creating Python 3.12 environment
- Installing required packages (PyTorch, transformers, etc.)
- Downloading LaMA inpainting model (~200MB)

In [ ]:
# Setup environment directly
print("🔧 Setting up environment...\n")

# Create conda environment from environment.yml
print("📦 Creating conda environment...")
!conda env create -f environment.yml

# Activate and install additional dependencies
print("\n📦 Installing additional packages...")
!conda run -n py312aiwatermark pip install PyQt6 transformers iopaint opencv-python-headless

# Download the LaMA model
print("\n📥 Downloading LaMA model (this may take a few minutes)...")
!conda run -n py312aiwatermark iopaint download --model lama

print("\n✅ All dependencies installed!")
print("✅ LaMA model downloaded!")
print("\n🎉 Setup complete! Ready to process videos.")


## Step 4: Upload Your SORA 2 Video

Click the **"Choose Files"** button below to upload your video.

**Supported formats:** MP4, AVI, MOV, MKV

**Note:** Large files may take a while to upload. Be patient!


In [ ]:
from google.colab import files
import os

print("Please select your SORA 2 video file...\n")
uploaded = files.upload()

# Get the uploaded filename
input_video = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {input_video}")
print(f"📊 File size: {os.path.getsize(input_video) / (1024*1024):.2f} MB")


In [ ]:
# ===== CUSTOMIZE THESE SETTINGS =====

# Detection sensitivity (1-100, lower = more aggressive)
max_bbox_percent = 15  # Good for SORA 2 watermarks

# Output format (MP4, AVI, WEBM, MOV)
output_format = "MP4"

# Make watermarks transparent instead of inpainting (slower)
transparent = False

# Output filename
output_video = "output_no_watermark.mp4"

# ====================================

print(" Configuration:")
print(f"   📊 Max BBox Percent: {max_bbox_percent}%")
print(f"   📹 Output Format: {output_format}")
print(f"   🎨 Transparent Mode: {transparent}")
print(f"   💾 Output File: {output_video}")
print("\n✅ Configuration ready!")


## 🚀 Step 6: Process Video

This is where the magic happens! The AI will:
1. Extract frames from your video
2. Detect watermarks in each frame
3. Remove watermarks using inpainting
4. Reassemble into a video

**Tip:** You'll see progress updates below. Don't close the tab!


In [ ]:
import time
import subprocess
import threading
import sys

start_time = time.time()

print("🚀 Starting video processing...\n")
print("="*60)

# Build the command - directly call Python script with conda
transparent_flag = "--transparent" if transparent else ""
overwrite_flag = "--overwrite"

# Make sure we're in the right directory
%cd /content/Sweeta

print(f"📹 Processing: {input_video}")
print(f"📁 Output: {output_video}")
print(f"⚙️ Settings: max_bbox_percent={max_bbox_percent}, format={output_format}")
print(f"🔧 Flags: {transparent_flag} {overwrite_flag}")
print("\n💡 This may take 5-15 minutes depending on video length...")
print("⏳ Processing frames (this will show progress when available)...\n")

# Build the full command
cmd = [
    "conda", "run", "-n", "py312aiwatermark", "python", "remwm.py",
    f"/content/Sweeta/{input_video}",
    f"/content/Sweeta/{output_video}",
    "--max-bbox-percent", str(max_bbox_percent),
    "--force-format", output_format,
]

# Add optional flags
if transparent:
    cmd.append("--transparent")
cmd.append("--overwrite")

print(f"🔍 Running command: {' '.join(cmd)}")
print("="*60)

# Run with real-time output
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Print output in real-time
try:
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
            sys.stdout.flush()
        
        # Check if process is taking too long (30 minutes timeout)
        if time.time() - start_time > 1800:  # 30 minutes
            print("\n⚠️ Process taking longer than 30 minutes. This might be normal for very long videos.")
            print("💡 You can wait longer or restart if needed.")
            break
            
except KeyboardInterrupt:
    print("\n🛑 Process interrupted by user")
    process.terminate()

# Wait for process to complete
return_code = process.poll()
if return_code is None:
    return_code = process.wait()

elapsed_time = time.time() - start_time
print("="*60)

if return_code == 0:
    print(f"\n✅ Processing complete!")
    print(f"⏱️ Time taken: {elapsed_time/60:.2f} minutes")
    print(f"📁 Output saved to: {output_video}")
    
    # Check if output file exists and show size
    import os
    if os.path.exists(output_video):
        size_mb = os.path.getsize(output_video) / (1024*1024)
        print(f"📊 Output file size: {size_mb:.2f} MB")
    else:
        print("⚠️ Warning: Output file not found!")
else:
    print(f"\n❌ Processing failed with return code: {return_code}")
    print("💡 Check the output above for error details")


## Step 7: Download Your Video

Your watermark-free video is ready! Click the download link below.


In [ ]:
# Check if processing is still running
import psutil
import os

print("🔍 System Status Check:")
print("="*50)

# Check GPU memory
print("📊 GPU Memory Usage:")
!nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

# Check if python processes are running
print("\n🐍 Python Processes:")
for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
    try:
        if 'python' in proc.info['name'].lower() and 'remwm' in ' '.join(proc.info['cmdline']):
            print(f"   PID {proc.info['pid']}: {' '.join(proc.info['cmdline'][:3])}")
    except:
        pass

# Check if output file is being created/updated
print(f"\n📁 Output File Status:")
if os.path.exists(output_video):
    size = os.path.getsize(output_video)
    print(f"   ✅ {output_video} exists ({size} bytes)")
    if size > 0:
        print("   📈 File is growing - processing is likely working!")
    else:
        print("   ⚠️ File is empty - processing may have just started")
else:
    print(f"   ❌ {output_video} not found yet")

# Check temp files (processing might create temporary files)
print(f"\n🗂️ Temporary Files:")
temp_files = [f for f in os.listdir('.') if f.startswith('tmp') or 'temp' in f.lower()]
if temp_files:
    for f in temp_files[:5]:  # Show first 5
        print(f"   📄 {f}")
else:
    print("   No temp files found")

print("\n💡 If you see GPU usage and/or temp files, processing is likely working!")
print("💡 Video processing can take 10-30+ minutes for longer videos.")


## 📥 Step 7: Download Your Video

Your watermark-free video is ready! Click the download link below.


In [ ]:
from google.colab import files
import os

if os.path.exists(output_video):
    print("📥 Downloading your video...\n")
    files.download(output_video)
    print("\n✅ Download started! Check your browser's download folder.")
    print(f"📊 Output file size: {os.path.getsize(output_video) / (1024*1024):.2f} MB")
else:
    print("❌ Error: Output file not found!")
    print("Please check the processing step for errors.")
    print("\n💡 Try running the status check cell above to see what's happening.")


## 🔧 Troubleshooting

### Common Issues:

**❌ "CUDA out of memory" error**
- Solution: Try processing a shorter video or restart runtime

**❌ "ModuleNotFoundError" or import errors**
- Solution: Re-run Step 3 (Install Dependencies)

**❌ Watermarks not fully removed**
- Solution: Try lowering `max_bbox_percent` to 10 or 5
- Alternative: Enable `transparent = True` mode

**❌ Video quality degraded**
- Solution: Try a different output format (MP4 → AVI)
- Check if original video had high quality

**❌ Processing takes too long**
- Note: Long videos take time. A 60s video can take 15+ minutes
- Tip: Process shorter clips for faster results

### Still having issues?

- 🐛 [Open an issue on GitHub](https://github.com/kuberwastaken/sweeta/issues)
- 📧 Contact: [Twitter @kuberwastaken](https://twitter.com/kuberwastaken)

---


## About This Project

### Credits:
- Built by [Kuber Mehta](https://kuber.studio/)
- Based on WatermarkRemover-AI by D-Ogi
- Development journey: [journal.md](https://github.com/kuberwastaken/sweeta/blob/main/journal.md)

### Links:
[GitHub Repository](https://github.com/kuberwastaken/sweeta), [Documentation](https://github.com/kuberwastaken/sweeta#readme), [Twitter @kuberwastaken](https://twitter.com/kuberwastaken)

---

<div align="center">

**Use only for Educational and Research Purposes**

[Star on GitHub](https://github.com/kuberwastaken/sweeta) if you found this useful!

**License:** Apache 2.0 | **Use responsibly and ethically**

</div>
